<a href="https://colab.research.google.com/github/syedanida/Computer-Vision/blob/main/Part_1_Supervised_Contrastive_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install tensorflow-addons
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install scikit-learn
!pip install opencv-python
!pip install pillow
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import time
from sklearn.manifold import TSNE
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout, GlobalAveragePooling2D, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Hyperparameters
BATCH_SIZE = 128
IMG_SIZE = 32
PROJECTION_DIM = 128
TEMPERATURE = 0.1
EPOCHS = 20
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.5
NUM_CLASSES = 10

In [ ]:
# Load CIFAR-10 dataset
def load_cifar10_dataset():
    """Load and preprocess CIFAR-10 dataset"""
    print("Loading CIFAR-10 dataset...")

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # Convert labels to 1D arrays
    y_train = tf.squeeze(y_train)
    y_test = tf.squeeze(y_test)

    # Normalize pixel values to [0, 1]
    x_train = tf.cast(x_train, tf.float32) / 255.0
    x_test = tf.cast(x_test, tf.float32) / 255.0

    print(f"Training data shape: {x_train.shape}, Training labels shape: {y_train.shape}")
    print(f"Test data shape: {x_test.shape}, Test labels shape: {y_test.shape}")

    # Get class names
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

    return (x_train, y_train), (x_test, y_test), class_names

In [ ]:
# Create TensorFlow datasets
def create_datasets(x_train, y_train, x_test, y_test, batch_size=BATCH_SIZE):
    """Create TensorFlow datasets for training and testing"""

    # Create training dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    # Create test dataset
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_dataset = test_dataset.batch(batch_size)

    return train_dataset, test_dataset

In [ ]:
# Data augmentation function
def get_data_augmentation():
    """Create data augmentation model"""
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    ])
    return data_augmentation

In [ ]:
# Create ResNet-based encoder
def create_encoder():
    """Create ResNet-based encoder for image feature extraction"""
    # Use a simplified architecture for CIFAR-10
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    # Initial convolutional block
    x = Conv2D(64, 3, strides=1, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Residual blocks
    for filters in [64, 128, 256]:
        # First block with downsampling
        x = Conv2D(filters, 3, strides=2, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

        # Second block with skip connection
        residual = x
        x = Conv2D(filters, 3, strides=1, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv2D(filters, 3, strides=1, padding="same")(x)
        x = BatchNormalization()(x)
        x = tf.keras.layers.add([x, residual])
        x = Activation("relu")(x)

    # Global pooling
    x = GlobalAveragePooling2D()(x)

    # Output layer
    outputs = Dense(PROJECTION_DIM)(x)

    # Create model
    encoder = Model(inputs=inputs, outputs=outputs, name="encoder")

    return encoder

In [ ]:
# Create projection head for contrastive learning
def create_projection_head():
    """Create projection head for contrastive learning"""
    inputs = Input(shape=(PROJECTION_DIM,))
    x = Dense(PROJECTION_DIM)(inputs)
    x = Activation("relu")(x)
    outputs = Dense(PROJECTION_DIM)(x)

    projection_head = Model(inputs, outputs, name="projection_head")
    return projection_head

In [ ]:
# Create classification head
def create_classification_head():
    """Create classification head"""
    inputs = Input(shape=(PROJECTION_DIM,))
    x = Dense(PROJECTION_DIM // 2)(inputs)
    x = Activation("relu")(x)
    x = Dropout(DROPOUT_RATE)(x)
    outputs = Dense(NUM_CLASSES, activation="softmax")(inputs)

    classification_head = Model(inputs, outputs, name="classification_head")
    return classification_head

In [ ]:
# Supervised Contrastive Loss function
class SupervisedContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, temperature=TEMPERATURE, name="supervised_contrastive_loss"):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors = tf.math.l2_normalize(feature_vectors, axis=1)

        # Get similarity matrix
        similarity_matrix = tf.matmul(feature_vectors, feature_vectors, transpose_b=True)

        # Scale by temperature
        similarity_matrix = similarity_matrix / self.temperature

        # Define labels equality
        labels = tf.reshape(labels, [-1, 1])
        mask_similar = tf.cast(tf.equal(labels, tf.transpose(labels)), tf.float32)

        # Create identity mask to exclude self-contrast
        batch_size = tf.shape(feature_vectors)[0]
        mask_self = tf.eye(batch_size)

        # Apply masks to exclude self-contrast and include only same-label pairs
        mask_positives = mask_similar * (1.0 - mask_self)
        mask_negatives = 1.0 - mask_self

        # Count number of positives per sample
        num_positives = tf.reduce_sum(mask_positives, axis=1)

        # Calculate log-probabilities
        exp_logits = tf.exp(similarity_matrix)
        log_prob = similarity_matrix - tf.math.log(tf.reduce_sum(exp_logits * mask_negatives, axis=1, keepdims=True))

        # Calculate mean of log-probabilities over positive samples
        mean_log_prob_pos = tf.reduce_sum(mask_positives * log_prob, axis=1) / (num_positives + 1e-8)

        # Return negative mean (for minimization)
        loss = -tf.reduce_mean(mean_log_prob_pos)

        return loss

In [ ]:
# Traditional classification model approach
def train_traditional_classification(x_train, y_train, x_test, y_test):
    """Train a traditional classification model"""
    print("\n--- Traditional Classification Approach ---\n")

    # Create datasets
    train_dataset, test_dataset = create_datasets(x_train, y_train, x_test, y_test)

    # Create data augmentation model
    data_augmentation = get_data_augmentation()

    # Create encoder
    encoder = create_encoder()

    # Create input layer
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    # Apply data augmentation
    x = data_augmentation(inputs)

    # Apply encoder
    features = encoder(x)

    # Apply classification head
    outputs = create_classification_head()(features)

    # Create model
    model = Model(inputs, outputs, name="traditional_classification_model")

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    # Create early stopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # Train model
    print("Training traditional classification model...")
    start_time = time.time()

    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=test_dataset,
        callbacks=[early_stopping]
    )

    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")

    # Evaluate model
    loss, accuracy = model.evaluate(test_dataset, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")

    return model, encoder, history, accuracy, training_time

In [ ]:
# Supervised contrastive learning approach
def train_supervised_contrastive(x_train, y_train, x_test, y_test):
    """Train using supervised contrastive learning approach"""
    print("\n--- Supervised Contrastive Learning Approach ---\n")

    # Create datasets
    train_dataset, test_dataset = create_datasets(x_train, y_train, x_test, y_test)

    # Create data augmentation model
    data_augmentation = get_data_augmentation()

    # Create encoder
    encoder = create_encoder()

    # Create projection head
    projection_head = create_projection_head()

    # Create early stopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # Phase 1: Contrastive pre-training
    print("Phase 1: Contrastive pre-training...")

    # Create pretraining model
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = data_augmentation(inputs)
    features = encoder(x)
    projections = projection_head(features)
    pretraining_model = Model(inputs, projections, name="pretraining_model")

    # Compile pretraining model
    pretraining_model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss=SupervisedContrastiveLoss(TEMPERATURE)
    )

    # Train pretraining model
    start_time = time.time()

    pretraining_history = pretraining_model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=test_dataset,
        callbacks=[early_stopping]
    )

    pretraining_time = time.time() - start_time
    print(f"Pretraining completed in {pretraining_time:.2f} seconds")

    # Phase 2: Classification training
    print("Phase 2: Classification training...")

    # Freeze encoder
    encoder.trainable = False

    # Create classification model
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    features = encoder(inputs)
    outputs = create_classification_head()(features)
    classification_model = Model(inputs, outputs, name="classification_model")

    # Compile classification model
    classification_model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    # Train classification model
    classification_start_time = time.time()

    classification_history = classification_model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=test_dataset,
        callbacks=[early_stopping]
    )

    classification_time = time.time() - classification_start_time
    print(f"Classification training completed in {classification_time:.2f} seconds")

    # Total training time
    total_training_time = pretraining_time + classification_time
    print(f"Total training time: {total_training_time:.2f} seconds")

    # Evaluate model
    loss, accuracy = classification_model.evaluate(test_dataset, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")

    # Create combined history
    combined_history = {
        'pretraining_loss': pretraining_history.history['loss'],
        'pretraining_val_loss': pretraining_history.history['val_loss'],
        'classification_loss': classification_history.history['loss'],
        'classification_val_loss': classification_history.history['val_loss'],
        'accuracy': classification_history.history['sparse_categorical_accuracy'],
        'val_accuracy': classification_history.history['val_sparse_categorical_accuracy'],
    }

    return classification_model, encoder, combined_history, accuracy, total_training_time

In [ ]:
def extract_embeddings(model, x_data, y_data, num_samples=1000):
    """Extract embeddings for visualization"""
    # Select random samples
    indices = np.random.choice(len(x_data), min(num_samples, len(x_data)), replace=False)

    # Use NumPy indexing directly
    x_samples = x_data[indices]
    y_samples = y_data[indices]

    # Create a model that outputs embeddings
    embedding_model = Model(
        inputs=model.input,
        outputs=model.layers[-2].output
    )

    # Extract embeddings
    embeddings = embedding_model.predict(x_samples)

    return embeddings, y_samples

In [ ]:
# Visualize embeddings with t-SNE
def visualize_embeddings(embeddings, labels, class_names, title):
    """Visualize embeddings using t-SNE"""
    print(f"Visualizing embeddings using t-SNE: {title}")

    # Apply t-SNE
    tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
    embeddings_2d = tsne.fit_transform(embeddings)

    # Create DataFrame for easier plotting
    df = pd.DataFrame({
        'x': embeddings_2d[:, 0],
        'y': embeddings_2d[:, 1],
        'label': labels
    })

    # Create plot
    plt.figure(figsize=(12, 10))

    # Plot each class with a different color
    for i, class_name in enumerate(class_names):
        mask = df['label'] == i
        plt.scatter(
            df.loc[mask, 'x'],
            df.loc[mask, 'y'],
            label=class_name,
            alpha=0.7,
            s=30
        )

    plt.title(title, fontsize=16)
    plt.legend(title="Classes", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.tight_layout()
    plt.savefig(f"{title.replace(' ', '_')}.png", dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
# Visualize training history
def visualize_training_history(traditional_history, contrastive_history):
    """Visualize training history for both approaches"""
    # Create figure
    plt.figure(figsize=(15, 6))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(
        traditional_history.history['sparse_categorical_accuracy'],
        label='Traditional (Train)'
    )
    plt.plot(
        traditional_history.history['val_sparse_categorical_accuracy'],
        label='Traditional (Val)'
    )
    plt.plot(
        contrastive_history['accuracy'],
        label='Contrastive (Train)'
    )
    plt.plot(
        contrastive_history['val_accuracy'],
        label='Contrastive (Val)'
    )
    plt.title('Accuracy Comparison', fontsize=14)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(alpha=0.3)

    # Plot loss (not directly comparable)
    plt.subplot(1, 2, 2)
    plt.plot(
        traditional_history.history['loss'],
        label='Traditional (Train)'
    )
    plt.plot(
        traditional_history.history['val_loss'],
        label='Traditional (Val)'
    )
    plt.plot(
        contrastive_history['classification_loss'],
        label='Contrastive Classification (Train)'
    )
    plt.plot(
        contrastive_history['classification_val_loss'],
        label='Contrastive Classification (Val)'
    )
    plt.title('Loss Comparison', fontsize=14)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig("training_history_comparison.png", dpi=300, bbox_inches="tight")
    plt.show()

    # Plot pretraining loss separately
    plt.figure(figsize=(8, 6))
    plt.plot(
        contrastive_history['pretraining_loss'],
        label='Train'
    )
    plt.plot(
        contrastive_history['pretraining_val_loss'],
        label='Validation'
    )
    plt.title('Supervised Contrastive Pretraining Loss', fontsize=14)
    plt.xlabel('Epoch')
    plt.ylabel('Contrastive Loss')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig("pretraining_loss.png", dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
# Compare model performance
def compare_performance(traditional_acc, contrastive_acc, traditional_time, contrastive_time):
    """Compare performance metrics between approaches"""
    # Create figure
    plt.figure(figsize=(12, 6))

    # Plot accuracy comparison
    plt.subplot(1, 2, 1)
    methods = ['Traditional', 'Supervised Contrastive']
    accuracies = [traditional_acc, contrastive_acc]

    bars = plt.bar(methods, accuracies, color=['#3498db', '#e74c3c'])
    plt.title('Test Accuracy Comparison', fontsize=14)
    plt.ylabel('Accuracy')
    plt.ylim(0, 1.0)

    # Add accuracy values on top of bars
    for bar, acc in zip(bars, accuracies):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.01,
            f"{acc:.4f}",
            ha='center',
            fontweight='bold'
        )

    # Plot training time comparison
    plt.subplot(1, 2, 2)
    times = [traditional_time, contrastive_time]

    bars = plt.bar(methods, times, color=['#3498db', '#e74c3c'])
    plt.title('Training Time Comparison', fontsize=14)
    plt.ylabel('Time (seconds)')

    # Add time values on top of bars
    for bar, t in zip(bars, times):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 20,
            f"{t:.0f}s",
            ha='center',
            fontweight='bold'
        )

    plt.tight_layout()
    plt.savefig("performance_comparison.png", dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
# Main function
def main():
    print("Supervised Contrastive Learning vs Traditional Classification")

    # Load CIFAR-10 dataset
    (x_train, y_train), (x_test, y_test), class_names = load_cifar10_dataset()

    # Train traditional classification model
    traditional_model, traditional_encoder, traditional_history, traditional_acc, traditional_time = train_traditional_classification(
        x_train, y_train, x_test, y_test
    )

    # Train supervised contrastive model
    contrastive_model, contrastive_encoder, contrastive_history, contrastive_acc, contrastive_time = train_supervised_contrastive(
        x_train, y_train, x_test, y_test
    )

    # Extract embeddings for visualization
    traditional_embeddings, traditional_labels = extract_embeddings(
        traditional_model, x_test, y_test
    )

    contrastive_embeddings, contrastive_labels = extract_embeddings(
        contrastive_model, x_test, y_test
    )

    # Visualize embeddings
    visualize_embeddings(
        traditional_embeddings, traditional_labels, class_names,
        "Traditional Classification Embeddings"
    )

    visualize_embeddings(
        contrastive_embeddings, contrastive_labels, class_names,
        "Supervised Contrastive Learning Embeddings"
    )

    # Visualize training history
    visualize_training_history(traditional_history, contrastive_history)

    # Compare performance
    compare_performance(
        traditional_acc, contrastive_acc,
        traditional_time, contrastive_time
    )

    print("\nExperiment completed!")
    print(f"Traditional Classification - Accuracy: {traditional_acc:.4f}, Training Time: {traditional_time:.2f}s")
    print(f"Supervised Contrastive Learning - Accuracy: {contrastive_acc:.4f}, Training Time: {contrastive_time:.2f}s")

In [ ]:
if __name__ == "__main__":
    main()

Supervised Contrastive Learning vs Traditional Classification
Loading CIFAR-10 dataset...
Training data shape: (50000, 32, 32, 3), Training labels shape: (50000,)
Test data shape: (10000, 32, 32, 3), Test labels shape: (10000,)

--- Traditional Classification Approach ---

Training traditional classification model...
Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 27s 48ms/step - loss: 1.7645 - sparse_categorical_accuracy: 0.3731 - val_loss: 1.4894 - val_sparse_categorical_accuracy: 0.4726
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 18s 47ms/step - loss: 1.2474 - sparse_categorical_accuracy: 0.5533 - val_loss: 1.4329 - val_sparse_categorical_accuracy: 0.5091
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 18s 47ms/step - loss: 1.0631 - sparse_categorical_accuracy: 0.6211 - val_loss: 1.1955 - val_sparse_categorical_accuracy: 0.6140
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - loss: 0.9376 - sparse_categorical_accuracy: 0.6697 - val_loss: 1.4860 - val_sparse_categorical_accuracy: 0.5624
Epoch 5/2

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([6366, 9628, 5916,  681, 5459, 4393, 7048, 7138,  876,  928, 8332,
       8359, 9149, 3784, 7859, 5296, 3579, 1087, 4352, 1611, 5102, 2618,
       1346,   53, 3256, 9348, 3978, 3752,  856, 3211, 7992, 5185, 2412,
       7203, 8497, 2910, 4401, 2370, 4918, 1827, 9518, 1279, 9925, 3277,
       2127, 9235, 4902, 7176, 9691, 7014, 6166, 4940, 5105, 4165, 9446,
       1692, 7431, 3077, 7343, 7219, 8117, 4279, 9935, 4688, 1170, 4095,
       4678, 9241, 6017, 4516, 2585,   93, 2133, 5408, 4780, 4219, 8672,
       5282,  141, 6016, 6732, 3769, 6410, 9033, 8944, 7777, 6982, 3948,
        118, 4194, 6202, 7122,  503,  787, 7429, 8145, 6546, 9138, 6555,
       2392, 5447, 3210, 3519,  859, 3416, 5643, 7322, 5071, 7266, 4309,
       2448, 2414, 6022, 9853, 1958, 8273, 4436,  728,  322, 6800, 7106,
       8488, 5061, 4392, 7542, 7422, 7094, 4072, 2864, 5655